In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
!pip install pydicom

In [18]:
!pip install imgaug

In [32]:
import pandas as pd
import numpy as np
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras import layers
from tensorflow.keras.models import Model
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pydicom as PDCM
from pydicom import dcmread
from pydicom.data import get_testdata_files
from tensorflow.keras.applications.vgg16 import preprocess_input
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from keras.models import Sequential
from keras import optimizers
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.7/dist-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['test']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [64]:
# Read Data
train = pd.read_csv(r"/content/drive/MyDrive/深度學習於醫學影像/HW2/hwk02_data_new/train.csv")
test = pd.read_csv(r"/content/drive/MyDrive/深度學習於醫學影像/HW2/hwk02_data_new/test.csv")

In [21]:
train

,ID,Age,Gender,Stage,FilePath,index
0,A159139,76,0,3,/DICOM/A159139/00010031,16
1,A653195,68,0,1,/DICOM/A653195/00010016,7
2,A152206,64,1,1,/DICOM/A152206/00010023,11
3,A691509,76,0,3,/DICOM/A691509/00010021,8
4,A533541,80,0,2,/DICOM/A533541/00010031,13
...,...,...,...,...,...,...
156,A608573,47,0,3,/DICOM/A608573/00010019,6
157,A340115,74,0,2,/DICOM/A340115/00010036,16
158,A671272,87,0,3,/DICOM/A671272/00010019,7
159,A605135,68,1,3,/DICOM/A605135/00010032,16


In [65]:
test

,ID,Age,Gender,FilePath,index,Stage 0,Stage 1,Stage 2,Stage 3,Stage
0,A518799,63,0,/DICOM/A518799/00010024,9,NaN,NaN,NaN,NaN,NaN
1,A653001,68,0,/DICOM/A653001/00010017,8,NaN,NaN,NaN,NaN,NaN
2,A199100,71,1,/DICOM/A199100/00010022,9,NaN,NaN,NaN,NaN,NaN
3,A131423,81,0,/DICOM/A131423/00010020,6,NaN,NaN,NaN,NaN,NaN
4,A565585,71,1,/DICOM/A565585/00010040,24,NaN,NaN,NaN,NaN,NaN
5,A478245,75,1,/DICOM/A478245/00010019,7,NaN,NaN,NaN,NaN,NaN
6,A101811,63,0,/DICOM/A101811/00010035,16,NaN,NaN,NaN,NaN,NaN
7,A656899,72,1,/DICOM/A656899/00010024,12,NaN,NaN,NaN,NaN,NaN
8,A570701,76,0,/DICOM/A570701/00010019,10,NaN,NaN,NaN,NaN,NaN
9,A679904,62,1,/DICOM/A679904/00010020,9,NaN,NaN,NaN,NaN,NaN


In [66]:
test = test.fillna(0)
test["Stage"] = 0

In [24]:
# *** CUT & JPG ***
len_train = len(train)
in_path = '/content/drive/MyDrive/深度學習於醫學影像/HW2/hwk02_data_new'
out_path = '/content/drive/MyDrive/深度學習於醫學影像/HW2/hwk02_data_new/jpg/'

# *** train ***
for i in range(len_train):
  #print(i)
  # Read DICOM
  index = train["index"][i]
  file_path = train["FilePath"][i]
  ds = dcmread(in_path+file_path) 
  # Set the Original Range
  pmax = ds.LargestImagePixelValue
  pmin = ds.SmallestImagePixelValue
  # Converted to Pixel Value 0~255
  rescale = (ds.pixel_array[index,40:90,40:90] - pmin) / (pmax - pmin) * 255
  # Uint8 Type
  image_uint8 = rescale.astype(np.uint8)
  # Show Image after Resize
  #fig = plt.figure()
  #fig.add_subplot(1, 2, 2).title.set_text('uint8')
  #plt.imshow(image_uint8)
  #plt.show()
  # Output JPG
  cv2.imwrite(out_path+train["ID"][i]+'.jpg', image_uint8)

len_test = len(test)
# *** test ***
for i in range(len_test):
  #print(i)
  # Read DICOM
  index = test["index"][i]
  file_path = test["FilePath"][i]
  ds = dcmread(in_path+file_path) 
  # Set the Original Range
  pmax = ds.LargestImagePixelValue
  pmin = ds.SmallestImagePixelValue
  # Converted to Pixel Value 0~255
  rescale = (ds.pixel_array[index,40:90,40:90] - pmin) / (pmax - pmin) * 255
  # Uint8 Type
  image_uint8 = rescale.astype(np.uint8)
  # Show Image after Resize
  #fig = plt.figure()
  #fig.add_subplot(1, 2, 2).title.set_text('uint8')
  #plt.imshow(image_uint8)
  #plt.show()
  # Output JPG
  cv2.imwrite(out_path+test["ID"][i]+'.jpg', image_uint8)

In [51]:
# Read images and merge them into the training data X.
X = np.array([])
path = '/content/drive/MyDrive/深度學習於醫學影像/HW2/hwk02_data_new/jpg/'
for i in range(len_train):
  input_path = path + train['ID'][i] + '.jpg'
  # Load image and Resize into (50, 50) 
  img = image.load_img(input_path, target_size=(50, 50))
  # Add one dimension and Become to (1, 224, 224, 3), which the last dimension is color.
  img2 = image.img_to_array(img)
  img2 = np.expand_dims(img2, axis=0)
  #print(img2.shape)
  if len(X.shape) == 1:
      X = img2
  else:
      # Combine the pixels of all images
      X = np.concatenate((X, img2), axis=0)

X = preprocess_input(X)
    
np.save('x_ray', X)

In [52]:
X.shape

(161, 50, 50, 3)

In [53]:
# Uneven Sample Distribution
Y = train["Stage"]
Counter(Y)

Counter({0: 22, 1: 22, 2: 42, 3: 75})

In [47]:
# SMOTE
X, Y = make_classification(n_classes=4, class_sep=1,
                          n_informative=2, n_redundant=0, flip_y=0,
                           n_features=50, n_clusters_per_class=1, n_samples=100,random_state=10)

sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X, Y)
print('Resampled dataset shape %s' % Counter(y_res))

Resampled dataset shape Counter({1: 25, 2: 25, 3: 25, 0: 25})


In [48]:
# Image Augmentation
data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.experimental.preprocessing.RandomRotation(0.1),
    ]
)

In [54]:
# Dummy Lable Dataframe
count = list(range(len_train))
df_label = pd.DataFrame(columns=['Stage 0','Stage 1','Stage 2','Stage 3'],index=count)
df_label = df_label.fillna(0)

# Dummy Lable
# 'Stage 0','Stage 1','Stage 2','Stage 3'
for i in range(len_train):
    row_label = train['Stage'][i]
    if row_label == 0:
      df_label["Stage 0"][i] = 1
    elif row_label == 1:
      df_label["Stage 1"][i] = 1
    elif row_label == 2:
      df_label["Stage 2"][i] = 1
    elif row_label == 3:
      df_label["Stage 3"][i] = 1

In [55]:
X.shape, y.shape

((161, 50, 50, 3), (161, 4))

In [56]:
# Set VGG16 Model
base_model = VGG16(weights='imagenet', include_top=False)

# Connect My Layer
x = base_model.output
#x = layers.Flatten()(x)
x = layers.GlobalAveragePooling2D()(x)
#x = layers.Dropout(0.2)(x)
x = layers.Dense(256, activation='relu')(x)
#x = layers.Dropout(0.2)(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dense(4, activation='sigmoid')(x)

# Set New Model inputs/outputs
model = Model(inputs=base_model.input, outputs=x)

# Original layers of VGG16 are not retrained
for layer in base_model.layers:
    layer.trainable = False
    
model.compile(optimizer='adam', loss='binary_crossentropy')

In [57]:
y = np.array(df_label)

In [59]:
model.fit(X, y, epochs=10, validation_split=0.2, verbose=2)

Epoch 1/10
4/4 - 5s - loss: 2.5276 - val_loss: 1.2637
Epoch 2/10
4/4 - 4s - loss: 1.0939 - val_loss: 1.5696
Epoch 3/10
4/4 - 4s - loss: 0.9765 - val_loss: 1.6988
Epoch 4/10
4/4 - 4s - loss: 0.8266 - val_loss: 1.2974
Epoch 5/10
4/4 - 4s - loss: 0.6581 - val_loss: 1.0880
Epoch 6/10
4/4 - 4s - loss: 0.4995 - val_loss: 0.8598
Epoch 7/10
4/4 - 4s - loss: 0.3984 - val_loss: 0.7749
Epoch 8/10
4/4 - 4s - loss: 0.3734 - val_loss: 0.7903
Epoch 9/10
4/4 - 4s - loss: 0.3253 - val_loss: 0.6983
Epoch 10/10
4/4 - 4s - loss: 0.2809 - val_loss: 0.6606


In [63]:
test

In [68]:
# Test 
path = '/content/drive/MyDrive/深度學習於醫學影像/HW2/hwk02_data_new/jpg/'
for n in range(len_test):
  print("n",n)
  output_path = path + test['ID'][n] + '.jpg'
  print(output_path)
  img = image.load_img(output_path, target_size=(50, 50))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)
  model_matrix = model.predict(x)
  # Model Matrix
  print("model_matrix = ",model_matrix)
  test['Stage 0'][n] = model_matrix[0][0]
  test['Stage 1'][n] = model_matrix[0][1]
  test['Stage 2'][n] = model_matrix[0][2]
  test['Stage 3'][n] = model_matrix[0][3]
  # Stage
  if max(model_matrix[0])== model_matrix[0][0]:
    test["Stage"][n] = 0
  elif max(model_matrix[0])== model_matrix[0][1]:
    test["Stage"][n] = 1
  elif max(model_matrix[0])== model_matrix[0][2]:
    test["Stage"][n] = 2
  elif max(model_matrix[0])== model_matrix[0][3]:
    test["Stage"][n] = 3

n 0
/content/drive/MyDrive/深度學習於醫學影像/HW2/hwk02_data_new/jpg/A518799.jpg
model_matrix =  [[0.04290909 0.14803094 0.3943509  0.00746685]]
n 1
/content/drive/MyDrive/深度學習於醫學影像/HW2/hwk02_data_new/jpg/A653001.jpg
model_matrix =  [[0.05231646 0.01410422 0.5986721  0.05494973]]
n 2
/content/drive/MyDrive/深度學習於醫學影像/HW2/hwk02_data_new/jpg/A199100.jpg
model_matrix =  [[0.05944833 0.06893447 0.8927984  0.04883519]]
n 3
/content/drive/MyDrive/深度學習於醫學影像/HW2/hwk02_data_new/jpg/A131423.jpg


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/i

model_matrix =  [[0.00633714 0.09354943 0.18517444 0.88581455]]
n 4
/content/drive/MyDrive/深度學習於醫學影像/HW2/hwk02_data_new/jpg/A565585.jpg
model_matrix =  [[0.30582348 0.09377563 0.4804204  0.00223204]]
n 5
/content/drive/MyDrive/深度學習於醫學影像/HW2/hwk02_data_new/jpg/A478245.jpg
model_matrix =  [[0.00449705 0.0547699  0.0878728  0.88194263]]
n 6
/content/drive/MyDrive/深度學習於醫學影像/HW2/hwk02_data_new/jpg/A101811.jpg


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


model_matrix =  [[5.2046776e-04 5.2671045e-02 5.5150598e-01 4.5731193e-01]]
n 7
/content/drive/MyDrive/深度學習於醫學影像/HW2/hwk02_data_new/jpg/A656899.jpg
model_matrix =  [[0.19899386 0.00395402 0.26456088 0.2590981 ]]
n 8
/content/drive/MyDrive/深度學習於醫學影像/HW2/hwk02_data_new/jpg/A570701.jpg
model_matrix =  [[0.0012545  0.00927937 0.00279987 0.9998268 ]]
n 9
/content/drive/MyDrive/深度學習於醫學影像/HW2/hwk02_data_new/jpg/A679904.jpg
model_matrix =  [[0.00445592 0.36018085 0.24118409 0.42878258]]
n 10
/content/drive/MyDrive/深度學習於醫學影像/HW2/hwk02_data_new/jpg/A75774.jpg
model_matrix =  [[0.00485206 0.05660328 0.27745754 0.88764876]]
n 11
/content/drive/MyDrive/深度學習於醫學影像/HW2/hwk02_data_new/jpg/X388459.jpg
model_matrix =  [[0.13643482 0.1801211  0.8466591  0.00106117]]
n 12
/content/drive/MyDrive/深度學習於醫學影像/HW2/hwk02_data_new/jpg/A652063.jpg
model_matrix =  [[0.01894629 0.02390522 0.1078513  0.9762271 ]]
n 13
/content/drive/MyDrive/深度學習於醫學影像/HW2/hwk02_data_new/jpg/A122221.jpg
model_matrix =  [[0.10203263 0.23

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


model_matrix =  [[0.01920873 0.05000243 0.22919646 0.26314527]]
n 16
/content/drive/MyDrive/深度學習於醫學影像/HW2/hwk02_data_new/jpg/A535518.jpg
model_matrix =  [[2.8845668e-04 8.3775640e-02 2.8011501e-03 9.9958956e-01]]
n 17
/content/drive/MyDrive/深度學習於醫學影像/HW2/hwk02_data_new/jpg/A561140.jpg
model_matrix =  [[0.01635471 0.0428929  0.5621722  0.5759112 ]]
n 18
/content/drive/MyDrive/深度學習於醫學影像/HW2/hwk02_data_new/jpg/A348307.jpg
model_matrix =  [[0.00103956 0.01705131 0.07127813 0.99985313]]
n 19
/content/drive/MyDrive/深度學習於醫學影像/HW2/hwk02_data_new/jpg/A73567.jpg
model_matrix =  [[0.10780087 0.07769245 0.5126596  0.00810158]]
n 20
/content/drive/MyDrive/深度學習於醫學影像/HW2/hwk02_data_new/jpg/A11766.jpg
model_matrix =  [[0.00697017 0.5703572  0.02273762 0.8359891 ]]
n 21
/content/drive/MyDrive/深度學習於醫學影像/HW2/hwk02_data_new/jpg/A249298.jpg
model_matrix =  [[0.0516772  0.18374294 0.2189757  0.12203968]]
n 22
/content/drive/MyDrive/深度學習於醫學影像/HW2/hwk02_data_new/jpg/A305658.jpg
model_matrix =  [[3.9851665e-04

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


model_matrix =  [[0.01020899 0.01369163 0.03876457 0.98562086]]
n 31
/content/drive/MyDrive/深度學習於醫學影像/HW2/hwk02_data_new/jpg/A48100.jpg
model_matrix =  [[0.00575119 0.20774838 0.17552161 0.97738385]]
n 32
/content/drive/MyDrive/深度學習於醫學影像/HW2/hwk02_data_new/jpg/A109895.jpg
model_matrix =  [[0.06568515 0.03091767 0.05102682 0.47352824]]
n 33
/content/drive/MyDrive/深度學習於醫學影像/HW2/hwk02_data_new/jpg/A620420.jpg
model_matrix =  [[0.02921382 0.06162211 0.80553895 0.11522746]]
n 34
/content/drive/MyDrive/深度學習於醫學影像/HW2/hwk02_data_new/jpg/A646753.jpg
model_matrix =  [[0.01014853 0.07339618 0.36378193 0.43541813]]
n 35
/content/drive/MyDrive/深度學習於醫學影像/HW2/hwk02_data_new/jpg/A648480.jpg
model_matrix =  [[0.00473985 0.04711393 0.15191737 0.97024393]]
n 36
/content/drive/MyDrive/深度學習於醫學影像/HW2/hwk02_data_new/jpg/A629589.jpg
model_matrix =  [[0.03709906 0.15483749 0.11136061 0.06841642]]
n 37
/content/drive/MyDrive/深度學習於醫學影像/HW2/hwk02_data_new/jpg/A217497.jpg
model_matrix =  [[0.00332135 0.16653281 0.

In [69]:
test

,ID,Age,Gender,FilePath,index,Stage 0,Stage 1,Stage 2,Stage 3,Stage
0,A518799,63,0,/DICOM/A518799/00010024,9,0.042909,0.148031,0.394351,0.007467,2
1,A653001,68,0,/DICOM/A653001/00010017,8,0.052316,0.014104,0.598672,0.054950,2
2,A199100,71,1,/DICOM/A199100/00010022,9,0.059448,0.068934,0.892798,0.048835,2
3,A131423,81,0,/DICOM/A131423/00010020,6,0.006337,0.093549,0.185174,0.885815,3
4,A565585,71,1,/DICOM/A565585/00010040,24,0.305823,0.093776,0.480420,0.002232,2
5,A478245,75,1,/DICOM/A478245/00010019,7,0.004497,0.054770,0.087873,0.881943,3
6,A101811,63,0,/DICOM/A101811/00010035,16,0.000520,0.052671,0.551506,0.457312,2
7,A656899,72,1,/DICOM/A656899/00010024,12,0.198994,0.003954,0.264561,0.259098,2
8,A570701,76,0,/DICOM/A570701/00010019,10,0.001254,0.009279,0.002800,0.999827,3
9,A679904,62,1,/DICOM/A679904/00010020,9,0.004456,0.360181,0.241184,0.428783,3


In [70]:
# Save VGG16 result
test.to_csv("/content/drive/MyDrive/深度學習於醫學影像/HW2/VGG16.csv")  

In [ ]:
# Restnet
restnet = ResNet50(include_top=False, weights='imagenet', input_shape=(50,50,3))
output = restnet.layers[-1].output
output = keras.layers.Flatten()(output)
restnet = Model(restnet.input, outputs=output)
for layer in restnet.layers:
    layer.trainable = False


model = Sequential()
model.add(restnet)
model.add(Dense(512, activation='relu', input_dim=3))
model.add(Dropout(0.3))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(4, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['accuracy'])
model.summary()

In [ ]:
# Test 
for n in range(len_test):
  print("n",n)
  output_path = path + test['ID'][n] + '.jpg'
  print(output_path)
  img = image.load_img(output_path, target_size=(50, 50))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)
  model_matrix = model.predict(x)
  # Model Matrix
  print("model_matrix = ",model_matrix)
  test['Stage 0'][n] = model_matrix[0][0]
  test['Stage 1'][n] = model_matrix[0][1]
  test['Stage 2'][n] = model_matrix[0][2]
  test['Stage 3'][n] = model_matrix[0][3]
  # Stage
  if max(model_matrix[0])== model_matrix[0][0]:
    test["Stage"][n] = 0
  elif max(model_matrix[0])== model_matrix[0][1]:
    test["Stage"][n] = 1
  elif max(model_matrix[0])== model_matrix[0][2]:
    test["Stage"][n] = 2
  elif max(model_matrix[0])== model_matrix[0][3]:
    test["Stage"][n] = 3

In [ ]:
# Save VGG16 result
test.to_csv("/content/drive/MyDrive/深度學習於醫學影像/HW2/ResNet50.csv")  